# ATCNET
### importing libraries

In [1]:
#!pip install --upgrade torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 32.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 4.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 4.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.19.3
    Uninstalling nvidia-nccl-cu12-2.19.3:
      Successfully uninstalled nvidia-nccl-cu12-2.19.3
  Attempting uninstall: torch
    Found existing installation: torch 2.2.0
    Uninstalling torch-2.2.0:
      Successfully uninstalled torch-2.2.0


In [1]:
from braindecode.datasets import MOABBDataset
from numpy import multiply
from braindecode.preprocessing import(Preprocessor, exponential_moving_standardize, preprocess)
from braindecode.preprocessing import create_windows_from_events
import torch
from braindecode.models import ATCNet
from braindecode.util import set_random_seeds
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.lines import Line2D
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
import time
from braindecode import EEGClassifier
from sklearn.metrics import confusion_matrix
from braindecode.visualization import plot_confusion_matrix
from skorch.callbacks import EarlyStopping
import sys

/home/mt0/22CS60R61/miniconda3/envs/varnita/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/mt0/22CS60R61/miniconda3/envs/varnita/lib/python3.10/site-packages/torch/cuda/__init__.py:749: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen 

### importing dataset

In [2]:
subject_id = [1, 2, 3, 4, 5, 6, 7, 8, 9]
dataset = MOABBDataset(dataset_name = "BNCI2014001", subject_ids = subject_id)

BNCI2014001 has been renamed to BNCI2014_001. BNCI2014001 will be removed in version 1.1.
The dataset class name 'BNCI2014001' must be an abbreviation of its code 'BNCI2014-001'. See moabb.datasets.base.is_abbrev for more information.


### preprocessing dataset

In [3]:

# low_cut_hz = 0.5
# high_cut_hz = 150
factor_new = 1e-3
init_block_size = 1000
factor = 1e6

preprocessors =[
    Preprocessor('pick_types', eeg = True, meg = False, stim = False),
    Preprocessor(lambda data: multiply(data, factor)),
    # Preprocessor('filter', l_freq = low_cut_hz, h_freq = high_cut_hz),
    Preprocessor(exponential_moving_standardize, factor_new = factor_new,
                 init_block_size = init_block_size)
]

preprocess(dataset, preprocessors, n_jobs = -1)

/home/mt0/22CS60R61/miniconda3/envs/varnita/lib/python3.10/site-packages/braindecode/preprocessing/preprocess.py:55: UserWarning: Preprocessing choices with lambda functions cannot be saved.
  warn('Preprocessing choices with lambda functions cannot be saved.')
/home/mt0/22CS60R61/miniconda3/envs/varnita/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/mt0/22CS60R61/miniconda3/envs/varnita/lib/python3.10/site-packages/torch/cuda/__init__.py:749: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCou

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/home/mt0/22CS60R61/miniconda3/envs/varnita/lib/python3.10/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(
2024-04-28 03:17:51.541384: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/home/mt0/22CS60R61/miniconda3/envs/varnita/lib/python3.10/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


...

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/home/mt0/22CS60R61/miniconda3/envs/varnita/lib/python3.10/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/home/mt0/22CS60R61/miniconda3/envs/varnita/lib/python3.10/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(
/home/mt0/22CS60R61/miniconda3/envs/varnita/lib/python3.10/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(
/home/mt0/22CS60R61/miniconda3/envs/varnita/lib/python3.10/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).

In [7]:
trial_start_offset_seconds = -0.5

sfreq = dataset.datasets[0].raw.info['sfreq']
assert all([ds.raw.info['sfreq'] == sfreq for ds in dataset.datasets])
trial_start_offset_samples = int(trial_start_offset_seconds *  sfreq)
windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples = trial_start_offset_samples,
    trial_stop_offset_samples = 0,
    preload = True
)

splitted = windows_dataset.split('session')
train_set = splitted['0train']
valid_set = splitted['1test']


Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']


Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']

In [4]:
cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'
if cuda:
  print("GPU Available")
  torch.backends.cudnn.benchmark = True
else:
  print("GPU not available")

GPU not available


In [9]:
seed = 20200220
set_random_seeds(seed = seed, cuda = cuda)

n_classes = 4
classes = list(range(n_classes))
n_chans = train_set[0][0].shape[0]
input_window_samples = train_set[0][0].shape[1]



model = ATCNet(
    n_chans,
    n_classes, conv_block_dropout=0.4, att_dropout=0.6, tcn_dropout=0.5)

print(model)

if cuda:
  model = model.cuda()

/home/mt0/22CS60R61/miniconda3/envs/varnita/lib/python3.10/site-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +
/home/mt0/22CS60R61/miniconda3/envs/varnita/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1031.)
  return F.conv2d(input, weight, bias, self.stride,


Layer (type (var_name):depth-idx)                       Input Shape               Output Shape              Param #                   Kernel Shape
ATCNet (ATCNet)                                         [1, 22, 1125]             [1, 4]                    --                        --
├─Ensure4d (ensuredims): 1-1                            [1, 22, 1125]             [1, 22, 1125, 1]          --                        --
├─Rearrange (dimshuffle): 1-2                           [1, 22, 1125, 1]          [1, 1, 1125, 22]          --                        --
├─_ConvBlock (conv_block): 1-3                          [1, 1, 1125, 22]          [1, 32, 20, 1]            --                        --
│    └─Conv2d (conv1): 2-1                              [1, 1, 1125, 22]          [1, 16, 1125, 22]         1,024                     [64, 1]
│    └─BatchNorm2d (bn1): 2-2                           [1, 16, 1125, 22]         [1, 16, 1125, 22]         32                        --
│    └─Conv2d (conv2): 2-3

In [10]:
log_file_path = "./output/atcnet/training_log2copy22.txt"
sys.stdout = open(log_file_path,"w")
start_time = time.time()

In [11]:
lr = 0.0009
weight_decay = 0
start_time = time.time()

batch_size = 60
n_epochs = 10

early_stopping = EarlyStopping(patience = 100, monitor = 'valid_loss', lower_is_better = True,  threshold=0.0001)

clf = EEGClassifier(
    model,
    criterion = torch.nn.NLLLoss,
    optimizer = torch.optim.AdamW,
    train_split = predefined_split(valid_set),
    optimizer__lr = lr,
    optimizer__weight_decay = weight_decay,
    batch_size = batch_size,
    callbacks = ["accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max = n_epochs - 1)), early_stopping],
    device = device,
    classes = classes
)

In [12]:
_ = clf.fit(train_set, y = None, epochs = n_epochs)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")
sys.stdout.close()

In [ ]:
# ===============================================Plotting ==========================================
results_columns = ['train_loss', 'valid_loss', 'train_accuracy', 'valid_accuracy']
df = pd.DataFrame(clf.history[:, results_columns], columns = results_columns,
                  index = clf.history[:, 'epoch'])
df = df.assign(train_misclass = 100 - 100 * df.train_accuracy,
               valid_misclass = 100 - 100 * df.valid_accuracy)

fig, ax1 = plt.subplots(figsize = (8, 3))
df.loc[:, ['train_loss', 'valid_loss']].plot(
    ax = ax1, style = ['-', ':'], marker = 'o', color ='tab:blue', legend = False, fontsize = 14)
ax1.tick_params(axis = 'y', labelcolor = 'tab:blue', labelsize = 14)
ax1.set_ylabel("Loss", color = 'tab:blue', fontsize = 14)

ax2 = ax1.twinx()
df.loc[:, ['train_misclass', 'valid_misclass']].plot(
    ax = ax2, style = ['-', ':'], marker = 'o', color = 'tab:red', legend = False)

ax2.tick_params(axis = 'y', labelcolor = 'tab:red', labelsize = 14)
ax2.set_ylabel("Misclassification Rate [%]", color ='tab:red', fontsize = 14)
ax2.set_ylim(ax2.get_ylim()[0], 85)
ax1.set_xlabel("Epoch", fontsize = 14)

handlers = []
handlers.append(Line2D([0], [0], color ='black', linewidth = 1, linestyle = '-', label = 'Train'))
handlers.append(Line2D([0], [0], color = 'black', linewidth = 1, linestyle = ':', label = 'Valid'))
plt.legend(handlers, [h.get_label() for h in handlers], fontsize = 14)
plt.tight_layout()
plt.savefig('./output/atcnet/Lossgraph2copy2.png')

In [ ]:
y_pred = clf.predict(valid_set)

confusion_mat = confusion_matrix(y_true, y_pred)

label_dict = windows_dataset.datasets[0].window_kwargs[0][1]['mapping']

labels = [k for k, v in sorted(label_dict.items(), key = lambda kv: kv[1])]

fig = plot_confusion_matrix(confusion_mat, class_names = labels, figsize = (30, 15))
fig.savefig('./output/atcnet/confusion_mat2copy2.png')